Coding a Predict model to predict Iris plants into three species

In [59]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Importing train dataset and taking a first look at it

dataset = pd.read_csv('iris.csv')

dataset.info() # There is no column with na values

In [61]:
# Taking a look at the first lines of the dataset
# According to Kaggle, the column species is the one that we gonna use to train our predict model
print(dataset.head())
dataset.groupby(['species']).count()

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [62]:
# Removing columns that will not be used for model training, those will be stored as X and y, y beeing the target values
X = dataset.drop(['species'], axis=1).values
y = dataset['species'].values

# Separating dataset into train and test variables. Since it is a small amount of data, we will consider 30% as test size.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Creating an instance of the predicting model KNeighborsClassifier.

In [63]:
# Getting the best number of neighbors:
from sklearn.model_selection import cross_val_score

k_values = range(1, 21)  # Testar n_neighbors de 1 a 20

# List to store results
scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    # Cross validation with 5 divisions
    score = cross_val_score(knn, X, y, cv=5, scoring='accuracy').mean()
    scores.append(score)

# Find the k value with best performance
best_k = k_values[np.argmax(scores)]
print(f"Best value for n_neighbors: {best_k}") # 6

Best value for n_neighbors: 6


In [64]:
# Creating instance of KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=6)

# Fitting out model using the train variables
knn.fit(X_train, y_train)

# Predicting our data from the test variables
knn_prediction = knn.predict(X_test)

In [65]:
# What was the Accuracy?
knn_accuracy = accuracy_score(y_test, knn_prediction)
print(f'Accuracy for KNeighbors: {format(np.round(knn_accuracy * 100, 2))}%')

Accuracy for KNeighbors: 88.89%


We can see that KNeighbors had a 95.56% Accuracy performance! Lets do it on a test dataset